# EchoPro Python Workflow

## Import libraries and configure the Jupyter notebook

In [ ]:
import matplotlib.pyplot as plt

# Python EchoPro
import EchoPro

# Allows us to grab the SemiVariogram class so we can use its models
from EchoPro.computation import SemiVariogram as SV

# obtain all visualization routines
from EchoPro.visualization import plot_layered_points, plot_kriging_results

# Allows us to easily use matplotlib widgets in our Notebook
%matplotlib widget

EchoPro.__version__

## Set up  EchoPro for a specific survey year

### Initialize EchoPro object  using configuration files

* `initialization_config.yml` -- parameters independent of survey year
* `survey_year_2019_config.yml` -- parameters specific to survey year
* `source` -- Define the region of data to use e.g. US, CAN, US & CAN
* `exclude_age1` -- States whether age 1 hake should be included in analysis.

In [ ]:
%%time
survey_2019 = EchoPro.Survey(init_file_path='../config_files/initialization_config.yml',
                             survey_year_file_path='../config_files/survey_year_2019_config.yml',
                             source=3, 
                             exclude_age1=True)

### Load and process input data 
* This data is stored in `survey_2019`

In [ ]:
%%time 
survey_2019.load_survey_data() #file_type='biological')

### Compute the areal biomass density
* The areal biomass density is stored in `survey_2019.bio_calc.transect_results_gdf` as `biomass_density_adult`

In [ ]:
%%time
survey_2019.compute_transect_results()

In [ ]:
print(f"Total Biomass Estimate without Kriging: {1e-6*survey_2019.bio_calc.transect_results_gdf.biomass_adult.sum():.3f} kmt")
print(f"Total Male Biomass Estimate without Kriging: {1e-6*survey_2019.bio_calc.transect_results_male_gdf.biomass_adult.sum():.3f} kmt")
print(f"Total Female Biomass Estimate without Kriging: {1e-6*survey_2019.bio_calc.transect_results_female_gdf.biomass_adult.sum():.3f} kmt")

In [ ]:
# reports = survey_2019.get_reports()

In [ ]:
# reports.create_and_write_reports(output_path="../../EchoPro_python_output")

## Obtain Kriging Mesh Data

### Access Kriging mesh object
* Reads mesh data files specified by `survey_2019` 

In [ ]:
krig_mesh = survey_2019.get_kriging_mesh()

### Apply coordinate transformations
* Longitude transformation
* Lat/Lon to distance

#### Transect points

In [ ]:
krig_mesh.apply_coordinate_transformation(coord_type='transect')

#### Mesh points

In [ ]:
krig_mesh.apply_coordinate_transformation(coord_type='mesh')

## Perform Ordinary Kriging of biomass density

* transformed mesh points
* semi-variogram model
* areal biomass density 

### Initialize Kriging routine

In [ ]:
kriging_params = dict(
    # kriging parameters
    k_max=10,
    k_min=3,
    R=0.0226287,
    ratio=0.001,
    
    # parameters for semi-variogram model
    s_v_params={'nugget': 0.0, 'sill': 0.95279, 'ls': 0.0075429,
                'exp_pow': 1.5, 'ls_hole_eff': 0.0},
    
    # grab appropriate semi-variogram model
    s_v_model=SV.generalized_exp_bessel
)

# initalize kriging routine
krig = survey_2019.get_kriging(kriging_params)

### Perform Kriging

* Generates total biomass at mesh points

In [ ]:
%%time
krig.run_biomass_kriging(krig_mesh)

In [ ]:
# compute variables at the Kriging mesh points
krig.compute_kriging_variables()

## Generate transect and Kriging based reports

In [ ]:
reports = survey_2019.get_reports()

In [ ]:
# TODO: figure out the best input for output_path 
reports.create_and_write_reports(output_path="../../EchoPro_python_output")

In [ ]:
# survey_2019.bio_calc.len_age_dist_all.sum("stratum")